# Panorama Image Stitching
You will generate a stitched panorama image from multiple input images.


In [ ]:
# Load required Libraries
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os

## Read in the images



In [ ]:
# read images
path = 'images'
input_path = path + '/set_1/'
filenames = [input_path + filename for filename in os.listdir(input_path)]
raw_images = [cv2.imread(filename) for filename in filenames]
count = len(raw_images)

## Reorder the images
As this assignment will require the merge of multiple images, we need to do some alignment of the images. THe idea is to reset the order of the images, and to make the centre image the "source" image, and to then extend the panorama to both sides. \

As this is not the key learning from this assignment, the code is provided, no modifications are required

In [ ]:
# reset the order of the images to make the center one is the source image, and extend to both sides
images = []
new_idx = (count - 1) // 2
k = -1
for i in range(count):
    new_idx = new_idx + k * i
    images.append(raw_images[new_idx])
    k *= -1

# initialize the source image
img_src = images[0]

## Perform panorama stitching

In this section, implement the panorama image stitching. Note that there are multiple approaches that will work to achieve this, and you can find multiple examples online. In this assignment, we will work through one possible implementation.

Note that in this example, the goal is to merge multiple images (many demos online only merge two images). Also, for this example, we will consider horizontal merges only. It is relatively straight forward to extend this implementation to consider vertical panorama stitching too.

Note: with this implementation, the quality varies with the number of input images (merging 2 images is ok, but for more than 2, the perspective isn't very good). So try varying the number of images used in the merge

In [ ]:
# the image need to stitch
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
for i in range(1, len(images)):
    img_dst = images[i]
    # make sure to merge from left to right
    if i % 2 == 0:
        img_src, img_dst = img_dst, img_src

    # detects keypoints and computes the descriptors for "source" and "destination" images (hint: use ORB features)
    kp1, des1 = orb.detectAndCompute(img_src, None)
    kp2, des2 = orb.detectAndCompute(img_dst, None)
    # create a brute foce matching function, and calculate the matches
    matches = bf.match(des1, des2)

    # sort the matches by distance
    matches = sorted(matches, key = lambda x: x.distance)

    # Keep only the top 50 matches
    matches = matches[:50]

    # Get the keypoints from the matches
    src_pts = np.zeros((len(matches), 2), dtype=np.float32).reshape(-1, 1, 2)
    dst_pts = np.zeros((len(matches), 2), dtype=np.float32).reshape(-1, 1, 2)
    for i, match in enumerate(matches):
        src_pts[i, :] = kp1[match.queryIdx].pt
        dst_pts[i, :] = kp2[match.trainIdx].pt

    # Compute the homography matrix using RANSAC
    M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    # show the keypoint matches (hint: use the opencv drawMatches function)
    image_matches = cv2.drawMatches(img_src, kp1, img_dst, kp2, matches, None)
    # show matched points image
    plt.figure(figsize = (15, 15))
    plt.imshow(image_matches)
    plt.axis('off')
    # get the height and width of the original images
    h1, w1, p1 = img_src.shape
    h2, w2, p2 = img_dst.shape
    h = np.maximum(h1, h2)
    w = np.maximum(w1, w2)

    move_dis = int(np.maximum(dst_pts[0][0][0], src_pts[0][0][0]))

    # apply perspective correction (hint: use opencv warpPerspective. Set the width and height to (w1 + w2 - move_dis, h))
    img_transform = cv2.warpPerspective(img_dst, M, (w1 + w2 - move_dis, h))
    # combine the source image to the transformed image
    img_transform[0:img_src.shape[0], 0:img_src.shape[1]] = img_src
    # use img_transform as the source image for the next iteration of the loop
    img_src = img_transform

In [ ]:
# Display final panorama
plt.figure(figsize = (15,15))
plt.axis('off')
plt.imshow(img_transform)

In [ ]:
def panorama_stitching(images, reduction = 0):
    # read images
    path = 'images'
    input_path = path + '/' + images + '/'
    filenames = [input_path + filename for filename in os.listdir(input_path)]
    raw_images = [cv2.imread(filename) for filename in filenames]
    count = len(raw_images)
    # reset the order of the images to make the center one is the source image, and extend to both sides
    images = []
    new_idx = (count - 1) // 2
    k = -1
    for i in range(count):
        new_idx = new_idx + k * i
        images.append(raw_images[new_idx])
        k *= -1

    # initialize the source image
    img_src = images[0]
    # the image need to stitch
    orb = cv2.ORB_create()
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    for i in range(1, len(images)):
        img_dst = images[i]
        # make sure to merge from left to right
        if i % 2 == 0:
            img_src, img_dst = img_dst, img_src

        # detects keypoints and computes the descriptors for "source" and "destination" images (hint: use ORB features)
        kp1, des1 = orb.detectAndCompute(img_src, None)
        kp2, des2 = orb.detectAndCompute(img_dst, None)
        # create a brute foce matching function, and calculate the matches
        matches = bf.match(des1, des2)

        # sort the matches by distance
        matches = sorted(matches, key = lambda x: x.distance)

        # Keep only the top 50 matches
        matches = matches[:50]

        # Get the keypoints from the matches
        src_pts = np.zeros((len(matches), 2), dtype=np.float32).reshape(-1, 1, 2)
        dst_pts = np.zeros((len(matches), 2), dtype=np.float32).reshape(-1, 1, 2)
        for i, match in enumerate(matches):
            src_pts[i, :] = kp1[match.queryIdx].pt
            dst_pts[i, :] = kp2[match.trainIdx].pt

        # Compute the homography matrix using RANSAC
        M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
        # show the keypoint matches (hint: use the opencv drawMatches function)
        image_matches = cv2.drawMatches(img_src, kp1, img_dst, kp2, matches, None)
        # show matched points image
        plt.figure(figsize = (15, 15))
        plt.imshow(image_matches)
        plt.axis('off')
        # get the height and width of the original images
        h1, w1, p1 = img_src.shape
        h2, w2, p2 = img_dst.shape
        h = np.maximum(h1, h2)
        w = np.maximum(w1, w2)

        move_dis = int(np.maximum(dst_pts[0][0][0], src_pts[0][0][0]))

        # apply perspective correction (hint: use opencv warpPerspective. Set the width and height to (w1 + w2 - move_dis, h))
        img_transform = cv2.warpPerspective(img_dst, M, (w1 + w2 - move_dis, h))
        # combine the source image to the transformed image
        img_transform[0:img_src.shape[0], 0:img_src.shape[1]] = img_src
        # use img_transform as the source image for the next iteration of the loop
        img_src = img_transform
        plt.figure(figsize = (15, 15))
        plt.imshow(img_transform)
        plt.axis('off')

In [ ]:
for i in range(1):
    panorama_stitching('set_2', i)

In [ ]:
for i in range(1):
    panorama_stitching('set_3', i)

In [ ]:
for i in range(1):
    panorama_stitching('set_4', i)

In [ ]:
for i in range(1):
    panorama_stitching('set_5', i)